# Problem statement

Let $X \in \{0,1\}$ be a random variable, that satisfies $$prob(X=1) = p = \frac {exp(a^{T}x+b)}{1+exp(a^{T}x+b)} $$  where $x \in \mathbb{R}^n$ is a vector of variables that affect the probability, and $a$ and $b$ are known parameters. We can think of $X = 1$ as the event that a consumer buys a product, and $x$ as a vector of variables that affect the probability, e.g., advertising effort, retail price, discounted price, packaging expense, and other factors. 
The variable $x$, which we are to optimize over, is subject to a set of linear constraints, $Fx \preceq g$.

Formulate the following problems as convex optimization problems.


1)  _Maximizing buying probability_ The goal is to choose $x$ to maximize $p$.

2)  _Maximizing expected profit._ Let $c^T x+d$ be the profit derived from selling the product, which we assume is positive for all feasible x. The goal is to maximize the expected profit, which is $p\cdot(c^T x + d)$.  


In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as sps
import cvxpy as cvx
%matplotlib inline